*ETL*

Cargar todas las librerías a utilizar:

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import ast
import pyarrow

Importar el CSV a Visual Code a través de Python

In [3]:
df = pd.read_csv(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\movies_dataset.csv', delimiter=',', quotechar='"')
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


Revisar el tipo de dato en cada columna del DataFrame para seleccionar cuáles son relevantes para el proyecto

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45463 non-null  float64
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Se pasa a int la columna budget y la columna ID que servirán para las futuras funciones.

In [5]:
# Reemplazar el valor '1997-08-20' con 0
df['budget'] = df['budget'].replace(pd.Timestamp('1997-08-20'), 0)

# Convertir la columna 'budget' a tipo int
df['budget'] = df['budget'].astype(int, errors='ignore')




In [6]:
# Reemplazar el valor '1997-08-20' con 0
df['id'] = df['id'].replace(pd.Timestamp('1997-08-20'), 0)

# Reemplazar NaT con 0
df['id'] = df['id'].fillna(0)


# Convertir la columna 'id' a tipo int
df['id'] = df['id'].astype(int, errors='ignore')

Identificar las variables que tienen datos anidados para darles el correcto tratamiento:
- belongs_to_collection
- genres
- production_companies
- production_coutries
- spoken_languages


In [7]:
def extract_full_collection_info(row):
    if pd.isna(row['belongs_to_collection']):
        return pd.Series([None, None, None, None], 
                         index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
    else:
        try:
            collection = json.loads(row['belongs_to_collection'].replace("'", "\""))
            return pd.Series([collection.get('id'), collection.get('name'), collection.get('poster_path'), collection.get('backdrop_path')], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
        except:
            return pd.Series([None, None, None, None], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])

# Aplicar la función a cada fila del DataFrame
full_collection_info = df.apply(extract_full_collection_info, axis=1)

# Añadir las nuevas columnas al DataFrame original
aux = pd.concat([df, full_collection_info],axis=1)

In [8]:

def genres_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def genres_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['genres_ids'] = df['genres'].apply(genres_ids)
df['genres_names'] = df['genres'].apply(genres_names)

df['genres_names_array'] = df['genres_names'].apply(lambda x: x if isinstance(x,list)else[])
aux5 = df[['genres_names_array']]                                                                              


In [9]:
aux5

,genres_names_array
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]
...,...
45458,"[Drama, Family]"
45459,[Drama]
45460,"[Action, Drama, Thriller]"
45461,[]


In [10]:

def extract_production_company_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_company_ids'] = df['production_companies'].apply(extract_production_company_ids)
df['production_company_names'] = df['production_companies'].apply(extract_production_company_names)

df['company_names_array'] = df['production_company_names'].apply(lambda x: x if isinstance(x,list)else[])
aux2 = df[['company_names_array']]                                                                              




In [11]:
aux2

,company_names_array
0,[Pixar Animation Studios]
1,"[TriStar Pictures, Teitler Film, Interscope Co..."
2,"[Warner Bros., Lancaster Gate]"
3,[Twentieth Century Fox Film Corporation]
4,"[Sandollar Productions, Touchstone Pictures]"
...,...
45458,[]
45459,[Sine Olivia]
45460,[American World Pictures]
45461,[Yermoliev]


In [12]:

def extract_production_company_iso(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['iso_3166_1'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_paises(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_countries_iso'] = df['production_countries'].apply(extract_production_company_iso)
df['production_countries_names'] = df['production_countries'].apply(extract_production_company_paises)

df['countries_names_array'] = df['production_countries_names'].apply(lambda x: x if isinstance(x,list)else[])
aux3 = df[['countries_names_array']]    



In [13]:
aux3

,countries_names_array
0,[United States of America]
1,[United States of America]
2,[United States of America]
3,[United States of America]
4,[United States of America]
...,...
45458,[Iran]
45459,[Philippines]
45460,[United States of America]
45461,[Russia]


In [14]:
def extract_iso_languages(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['iso_639_1'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_language_names(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['name'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []


# Aplicar las funciones a la columna 'spoken_languages'
df['spoken_languages_iso'] = df['spoken_languages'].apply(extract_iso_languages)
df['spoken_languages_name'] = df['spoken_languages'].apply(extract_language_names)

df['languages_names_array'] = df['spoken_languages_name'].apply(lambda x: x if isinstance(x,list)else[])
aux4 = df[['languages_names_array']]  

In [15]:
aux4

,languages_names_array
0,[English]
1,"[English, Français]"
2,[English]
3,[English]
4,[English]
...,...
45458,[فارسی]
45459,[]
45460,[English]
45461,[]


Uno los Dataframe nuevos al original

In [16]:
df_final = pd.concat([aux, aux2, aux3, aux4, aux5],axis=1)


In [17]:
df_final.shape

(45463, 32)

Se eliminan todas las columnas que no presentan utilidad para el posterior proceso de recomendación.

In [18]:
df_final=df_final.drop(columns=["belongs_to_collection","production_companies","production_countries","spoken_languages","genres","poster_path","tagline","collection_id","collection_poster_path"])

In [19]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int32  
 2   homepage                  7779 non-null   object 
 3   id                        45463 non-null  int32  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

Se reliza la conversión de ID y Budget para  utilizar posteriormente

In [20]:
df_final.rename(columns={'id':'movie_id'},inplace=True)

In [21]:
df_final['movie_id'] = df_final['movie_id'].fillna(0).astype(int)

In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int32  
 2   homepage                  7779 non-null   object 
 3   movie_id                  45463 non-null  int32  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

Se trasnforma en CSV para poder desarrollar una API luego.

In [23]:
df_final.to_csv(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\movies_output.csv', index= False)

Se realiza el ETL del archivo credits.csv

In [60]:
df_credits = pd.read_csv(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\credits.csv')


Se separa en dos el dataframe

In [25]:
# Crear df_credit_cast con las columnas "cast" y "id"
df_credit_cast = df_credits[['cast', 'id']].copy()

# Crear df_credit_crew con las columnas "crew" y "id"
df_credit_crew = df_credits[['crew','id']].copy()

Se cambia el nombre a movie_id para facilitar la comprensión del código. Luego se parsean las columnas anidadas

In [26]:
df_credit_cast.rename(columns={'id':'movie_id'},inplace=True)

In [27]:
df_credit_crew.rename(columns={'id':'movie_id'},inplace=True)

In [28]:
 
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}_')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast']

# Expandir las columnas de listas de diccionarios
df_credit_cast = expand_columns(df_credit_cast, columns_to_expand)

In [29]:
columns_to_expand = ['crew']
df_credit_crew = expand_columns(df_credit_crew, columns_to_expand)


Se eliminan columnas para poder crear un API luego.

In [30]:
df_credit_cast = df_credit_cast.drop(columns=["cast_cast_id","cast_credit_id","cast_id","cast_profile_path"])
#se eliminaron en total credit_id, order, profile_path y cast_credit_id


In [31]:
df_credit_crew = df_credit_crew.drop(columns=["crew_id"])


In [32]:
df_credit_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        564892 non-null  int64  
 1   cast_character  562474 non-null  object 
 2   cast_gender     562474 non-null  float64
 3   cast_name       562474 non-null  object 
 4   cast_order      562474 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 21.5+ MB


In [33]:
df_credit_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465085 entries, 0 to 465084
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           465085 non-null  int64  
 1   crew_credit_id     464314 non-null  object 
 2   crew_department    464314 non-null  object 
 3   crew_gender        464314 non-null  float64
 4   crew_job           464314 non-null  object 
 5   crew_name          464314 non-null  object 
 6   crew_profile_path  95098 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 24.8+ MB


In [57]:
crew_cast = pd.merge(df_credit_crew, df_credit_cast, on='movie_id', how='outer')

In [63]:
crew_cast.to_parquet(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\cast_crew.parquet')

In [35]:

output_file_path = r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\cast.parquet'
output_file_path1 = r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\crew.parquet'
df_credit_cast.to_parquet(output_file_path)
df_credit_crew.to_parquet(output_file_path1)

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0. Dataset: Movies

In [36]:
df_final['revenue'].fillna(0)
df_final['budget'].fillna(0)

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45458           0
45459           0
45460           0
45461           0
45462           0
Name: budget, Length: 45463, dtype: int32

Los valores nulos del campo release date deben eliminarse.


In [37]:
df_final['release_date'].dropna()



0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45457    1991-05-13
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45379, dtype: object

Convertir en tipo fecha los datos de la columna Date_Release con correspondiente manejo de errores

In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int32  
 2   homepage                  7779 non-null   object 
 3   movie_id                  45463 non-null  int32  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

In [39]:
df_final['release_date'].fillna('1900-01-01')


0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45458    1900-01-01
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [40]:
df_final['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45458           NaN
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [41]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'], errors='coerce')

Generar una nueva columna llamada "release_year" en donde figurará el año de estreno de la película.

In [42]:
df_final['release_year']=df_final['release_date'].dt.year


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.



In [43]:
df_final['budget'] = pd.to_numeric(df_final['budget'], errors='coerce').astype('Int64')


Cada columna tiene sus NaN reemplazados por ceros.

In [44]:
df_final['revenue'].fillna(0, inplace=True)
df_final['budget'].fillna(0, inplace=True)

Se realiza la cuenta para generar el KPI y se reemplazan los "inf" y "NaN" con ceros.

In [45]:
df_final['return'] = df_final['revenue'] / df_final['budget']
df_final['return'] = df_final['return'].replace(np.inf, 0).astype(float)
df_final['return'].fillna(0,inplace=True)



*DESARROLLO API*

Función 1: Se realiza una función en la que se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

In [46]:
def estrenos_por_mes(mes):
    meses = {
        'enero': 1,
        'febrero': 2,
        'marzo': 3,
        'abril': 4,
        'mayo': 5,
        'junio': 6,
        'julio': 7,
        'agosto': 8,
        'septiembre': 9,
        'octubre': 10,
        'noviembre': 11,
        'diciembre': 12
    }

    # Convertir el mes ingresado a minúsculas para asegurar coincidencia
    mes = mes.lower()
    
    # Obtener el número del mes correspondiente
    mes_num = meses.get(mes)
    
    if mes_num is None:
        return ValueError(f"Mes '{mes}' no reconocido. Por favor ingresa un mes válido.")
    
    # Filtrar el DataFrame por el mes
    filmaciones_mes = df_final[df_final['release_date'].dt.month == mes_num]
    
    # Devolver la cantidad de películas
    return len(filmaciones_mes)


In [47]:
estrenos_por_mes("julio")

2640

Función 2: Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [48]:
def estrenos_por_mes(dia):
    dias_semana = {
        'lunes': 1,
        'martes': 2,
        'miercoles': 3,
        'miércoles':4,
        'jueves':5,
        'viernes':6,
        'sabado':7,
        'domingo':8
   
    }

   
    # Obtener el número del mes correspondiente
    dia_num = dias_semana.get(dia)
    
    if dia_num is None:
        return ValueError(f"Día '{dia}' no reconocido. Por favor ingresa un mes válido.")
    
    # Filtrar el DataFrame por el mes
    filmaciones_dia = df_final[df_final['release_date'].dt.day == dia_num]
    
    # Devolver la cantidad de películas
    return len(filmaciones_dia)


In [49]:
estrenos_por_mes('jueves')

1289

Función 3:  Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.

In [50]:

data_final=df_final

# Convertir la columna release_date a datetime
df_final['release_date'] = pd.to_datetime(df_final['release_date'], errors='coerce')

# Verificar la conversión
print(df_final['release_date'])

# Función para obtener información de la película
def obtener_informacion_pelicula(titulo):
    # Filtrar el DataFrame por el título
    pelicula = df_final[df_final['title'].str.lower() == titulo.lower()]

    # Verificar si la película existe
    if pelicula.empty:
        return f"No se encontró ninguna película con el título '{titulo}'."
    
    # Obtener la información de la película
    titulo_pelicula = pelicula.iloc[0]['title']
    año_estreno = pelicula.iloc[0]['release_date'].year
    popularity = pelicula.iloc[0]['popularity']
    
    return {
        'Título': titulo_pelicula,
        'Año de Estreno': año_estreno,
        'Score': popularity
    }

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45458          NaT
45459   2011-11-17
45460   2003-08-01
45461   1917-10-21
45462   2017-06-09
Name: release_date, Length: 45463, dtype: datetime64[ns]


Función 4: Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [51]:

def obtener_informacion_pelicula(titulo):
    
    # Filtrar el DataFrame por el título
    pelicula = df_final[df_final['title'].str.lower() == titulo.lower()]

    # Verificar si la película existe
    if pelicula.empty:
        return f"No se encontró ninguna película con el título '{titulo}'."
    
    # Obtener la información de la película
    titulo_pelicula = pelicula.iloc[0]['title']
    vote_count = pelicula.iloc[0]['vote_count']
    vote_average= pelicula.iloc[0]['vote_average']

    if vote_count < 2000:
        return f"La película '{titulo_pelicula}' no cumple con el requisito de al menos 2000 valoraciones."

    
    return {
        'Título': titulo_pelicula,
        'Votos': vote_count,
        'Votos Promedio': vote_average,
    }

    

In [52]:
obtener_informacion_pelicula("Toy Story")

{'Título': 'Toy Story', 'Votos': 5415, 'Votos Promedio': 7.7}

Función 5 : Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.

In [53]:
def get_actor(nombre_actor):
    try:
        # Filtrar el DataFrame df_credit_cast para obtener las películas en las que ha participado el actor
        df_actor = df_credit_cast[df_credit_cast['cast_name'] == nombre_actor]
        
        # Verificar si df_actor está vacío
        if df_actor.empty:
            return f"El actor {nombre_actor} no se encontró en el dataset."

        # Asegurarnos de que la columna común se llama 'movie_id' en ambos DataFrames
        if 'movie_id' not in df_actor.columns or 'movie_id' not in df_final.columns:
            return "Error: La columna común 'movie_id' no existe en uno de los DataFrames."
        
        # Unir con df_movies usando la columna 'movie_id'
        df_actor_movies = pd.merge(df_actor, df_final, on='movie_id')
        
        # Eliminar duplicados basándonos en el título de la película
        df_actor_movies = df_actor_movies.drop_duplicates(subset=['title'])
        
        # Calcular el éxito del actor medido a través del retorno
        total_retorno = df_actor_movies['return'].sum()
        cantidad_peliculas = len(df_actor_movies)
        promedio_retorno = df_actor_movies['return'].mean()
        
        # Preparar el mensaje de resultado
        mensaje = (f"El actor {nombre_actor} ha participado de {cantidad_peliculas} filmaciones, "
                   f"con un retorno total de {total_retorno} y un promedio de {promedio_retorno:.2f} por filmación.")
        
        return mensaje
    except KeyError as e:
        return f"Error: La columna {e} no existe en el DataFrame."
    except Exception as e:
        return f"Error: {str(e)}"


In [54]:
get_actor("Tom Hanks")

'El actor Tom Hanks ha participado de 71 filmaciones, con un retorno total de 178.8494960552024 y un promedio de 2.52 por filmación.'

Función 6: Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.


In [55]:
def get_director(nombre_director):
    # Filtrar el DataFrame df_credit_crew para obtener las películas dirigidas por el director
    df_director = df_credit_crew[(df_credit_crew['crew_job'] == 'Director') & 
                                 (df_credit_crew['crew_name'] == nombre_director)]
    
    # Unir con df_movies usando la columna 'id'
    df_director_movies = pd.merge(df_director, df_final, on='movie_id')
    
    # Calcular el retorno (revenue - budget) y agregarlo al DataFrame
    df_director_movies['return'] = df_director_movies['revenue'] - df_director_movies['budget']
    
    # Eliminar duplicados basándonos en el título de la película
    df_director_movies = df_director_movies.drop_duplicates(subset=['title'])
    
    # Preparar el resultado
    resultado = []
    for _, row in df_director_movies.iterrows():
        titulo = row['title']
        fecha_lanzamiento = row['release_date']
        retorno_individual = row['return']
        costo = row['budget']
        ganancia = row['revenue']
        
        resultado.append({
            'Título': titulo,
            'Fecha de Lanzamiento': fecha_lanzamiento,
            'Retorno Individual': retorno_individual,
            'Costo': costo,
            'Ganancia': ganancia
        })
    
    return resultado


In [56]:
get_director("John Lasseter")

[{'Título': 'Toy Story',
  'Fecha de Lanzamiento': Timestamp('1995-10-30 00:00:00'),
  'Retorno Individual': 343554033,
  'Costo': 30000000,
  'Ganancia': 373554033},
 {'Título': "A Bug's Life",
  'Fecha de Lanzamiento': Timestamp('1998-11-25 00:00:00'),
  'Retorno Individual': 243258859,
  'Costo': 120000000,
  'Ganancia': 363258859},
 {'Título': 'Toy Story 2',
  'Fecha de Lanzamiento': Timestamp('1999-10-30 00:00:00'),
  'Retorno Individual': 407366869,
  'Costo': 90000000,
  'Ganancia': 497366869},
 {'Título': 'Luxo Jr.',
  'Fecha de Lanzamiento': Timestamp('1986-08-17 00:00:00'),
  'Retorno Individual': 0,
  'Costo': 0,
  'Ganancia': 0},
 {'Título': 'Cars',
  'Fecha de Lanzamiento': Timestamp('2006-06-08 00:00:00'),
  'Retorno Individual': 341983149,
  'Costo': 120000000,
  'Ganancia': 461983149},
 {'Título': 'Cars 2',
  'Fecha de Lanzamiento': Timestamp('2011-06-11 00:00:00'),
  'Retorno Individual': 359852396,
  'Costo': 200000000,
  'Ganancia': 559852396},
 {'Título': 'Tin Toy',